In [1]:
# Adaptation of https://elitedatascience.com/python-machine-learning-tutorial-scikit-learn
import dds
import sklearn
import pandas as pd
import logging
logging.basicConfig(level=logging.INFO)

import numpy as np
import pandas as pd
 
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import requests
import io
import json


In [2]:
%%sh
# rm -rf /tmp/data/*
# rm -rf /tmp/blobs/*

In [3]:
path_raw = "/wine-quality/raw"
path_model = "/wine-quality/my_model"
path_model_stats = "/wine-quality/my_model_stats.json"

def _load_data():
    print("*** in _load_data ***")
    url = "https://raw.githubusercontent.com/zygmuntz/wine-quality/master/winequality/winequality-red.csv"
    x = requests.get(url=url, verify=False).content 
    return pd.read_csv(io.StringIO(x.decode('utf8')), sep=";")


def data():
    return dds.keep(path_raw, _load_data)
# data = lambda: dds.keep(path_raw, _load_data)

In [4]:
data().head(3)

INFO:dds._api:Interaction tree:
INFO:dds._api:`- Fun <__main__._load_data> /wine-quality/raw <- ccd1ea31245b11507b79fdf02946f9438d7de3b6933ac87348ed8cb94a7bc0d5
INFO:dds.codec:Loading pandas codecs


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5


In [5]:
def build_model(X_train, y_train):
    print("*** in build_model ***")
    pipeline = make_pipeline(preprocessing.StandardScaler(), 
                             RandomForestRegressor(n_estimators=100))
    hyperparameters = { 'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
                      'randomforestregressor__max_depth': [None, 5, 3, 1]}

    clf = GridSearchCV(pipeline, hyperparameters, cv=10)
    
    clf.fit(X_train, y_train)
    return clf
 
    
def model_stats(clf, X_test, y_test) -> str:
    print("*** in model_stats ***")
    pred = clf.predict(X_test)
    return json.dumps({
#         "r2_score": r2_score(y_test, pred),
        "mse": mean_squared_error(y_test, pred)
    })
    
    
def pipeline():
    wine_data = data()
    y = wine_data.quality
    X = wine_data.drop('quality', axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=0.15, 
                                                        random_state=123, 
                                                        stratify=y)
    clf = dds.keep(path_model, build_model, X_train, y_train)
    dds.keep(path_model_stats, model_stats, clf, X_test, y_test)
    print("*** done ***")


dds.eval(pipeline, dds_export_graph="1.png")


INFO:dds._api:Interaction tree:
INFO:dds._api:`- Fun <__main__.pipeline> @ 229b55671940c51e042f2ac064aa2d3bfc8f2058927b8aa619ba1b9bb5077091
INFO:dds._api:   |- Dep path_model -> <__main__.path_model>: 4697524b4c751aacee319944ec2f0a265336dc96ff692488d7cbc071fc6a3733
INFO:dds._api:   |- Dep path_model_stats -> <__main__.path_model_stats>: 110ff33eb1f89810ce6af6621ae232ae55f5408eb739bd82972c361bffaa48da
INFO:dds._api:   |- Fun <__main__.data> @ 5e8a4da56caebb0454d220e6ba491b35d7200a317b410d6a549d9d49e911b6b8
INFO:dds._api:   |  |- Ctx 2af8fc03bd539aff5bbf9ccae9d82d7921223c8b858fe0c0f2d53f8260794f49
INFO:dds._api:   |  |- Dep path_raw -> <__main__.path_raw>: a1a3ce5966bea811120114207d004b9d61fea3f19ca024e5d8d0632fb90c40f4
INFO:dds._api:   |  `- Fun <__main__._load_data> /wine-quality/raw <- ccd1ea31245b11507b79fdf02946f9438d7de3b6933ac87348ed8cb94a7bc0d5
INFO:dds._api:   |     `- Ctx ba4c7d935aed88c22a170e6814c291cc3106a429cd443d7cf78c35a749ee1120
INFO:dds._api:   |- Fun <__main__.build_mo

*** done ***


In [6]:
%%sh
cat /tmp/data/wine-quality/my_model_stats.json

{"mse": 0.3352866666666666}

In [8]:

@dds.dds_function("/")

SyntaxError: unexpected EOF while parsing (<ipython-input-8-5b3ed9ef3202>, line 1)